# Description of all GVEC parameters

* We choose to use a dictionary of strings or array of strings for linebreaks. 
* If another parameter is mentioned, we use a string  `lnk_to_param(paramname)` or `lnk_to_param(linktext|paramname)` so that we can insert the link later. 
* the dictionary allows for (re-)formatting (either to markdown, with links, or to a screen output) and filtering the parameter list. 




In [ ]:
import yaml

dict = {}

# template:
# dict["paramname"] = {
#     "category": ["A","B"],
#     "subtitle": "[mandatory,essential,...] parameter"  # subtitle next to paramname
#     "required": True,  # sets "required input" marker
#     "description": [
#         "So it is, and so it goes, see lnk_to_param(here|param1) and lnk_to_param(`param2=4`|param2) and lnk_to_param(param3).",
#     ],
#     "required_if": " depends on ...",
#
#     "type": "[array of ][`integer`,`real`,`string`][of any size, of size x]"  ,
#     "allowed":'', #either write here what is allowed, like >0, or set the allowed values in the allowed_table
#     "allowed_table": [
#         ("`0`", "first option"),
#         ("`1`", "second option, see lnk_to_param(param4)"),
#     ],
#     "default":"`default value` or how the default depends on other variables",
# }

dict["whichInitEquilibrium"] = {
    "category": ["Initialization"],
    "subtitle": "mandatory",
    "required": True,
    "description": [
        "How initial guess is computed. Either from boundary and axis parameters, or from VMEC file.",
    ],
    # "required_if": "",
    # "default":"",
    # "allowed": '',
    "allowed_table": [
        ("`0`", "from axis and boundary parameters"),
        ("`1`", "from VMEC file, **needs:** lnk_to_param(VMECwoutfile)"),
    ],
}


dict["whichInitEquilibrium"] = {
    "category": ["Initialization"],
    "subtitle": "mandatory",
    "required": True,
    "description": [
        "How initial guess is computed. Either from boundary and axis parameters, or from VMEC file.",
    ],
    # "required_if": "",
    # "default":"",
    "type": "`integer`",
    # "allowed": '',
    "allowed_table": [
        ("`0`", "from axis and boundary parameters"),
        ("`1`", "from VMEC file, **needs:** lnk_to_param(VMECwoutfile)"),
    ],
}
dict["VMECwoutfile"] = {
    "category": ["Initialization", "VMEC"],
    # "subtitle":
    "description": [
        "full file name of vmec solution file, either as netcdf or as nemec output,",
        "**needs** lnk_to_param(VMECwoutfile_format)",
    ],
    "required_if": "lnk_to_param(`whichInitEquilibrium=1`|whichInitEquilibrium)",
    "type": "`string`",
    # "allowed": '',
    # "default":"",
}
dict["VMECwoutfile_format"] = {
    "category": ["Initialization", "VMEC"],
    # "subtitle":
    "description": [
        "Choose which VMEC wout file format, either netcdf or ascii/binary format from NEMEC",
        "The filename is specified in lnk_to_param(VMECwoutfile)",
    ],
    "required_if": "lnk_to_param(`whichInitEquilibrium=1`|whichInitEquilibrium)",
    "type": "`string`",
    "allowed_table": [
        ("`0`", "netcdf format"),
        ("`1`", "ascii format from older vmec (nemec)"),
        ("`2`", "binary format from older vmec (nemec)"),
    ],
    "default": "`0`",
}
for prof, proflong in zip(["iota", "pres"], ["rotational transform", "pressure"]):
    dict[f"{prof}_type"] = {
        "category": ["Initialization", "profiles"],
        "subtitle": "essential parameter",
        "description": [
            f"Type of profile to represent the {proflong} `{prof}(s)`. Can be a polynomial, B-Spline or a set of points which is then interpolated",
            "**Note:** the profile is always given as a function of the normalized magnetic flux $s$, with $s=0$ as the magnetic axis, and $s=1$ at the boundary. The normalized radius-like coordinate is "
            + r"$\rho=\sqrt{s}$.",
        ],
        "type": "`string`",
        "allowed_table": [
            (
                '`"polynomial"`',
                f"polynomial represenation, **needs** lnk_to_param({prof}_coefs)",
            ),
            (
                '`"bspline"`',
                f"B-Spline representation, **needs** lnk_to_param({prof}_coefs) and lnk_to_param({prof}_knots)",
            ),
            (
                '`"interpolation"`',
                f"Cubic spline interpolation from point values at s-positions, **needs** lnk_to_param({prof}_rho2) and lnk_to_param({prof}_vals)",
            ),
        ],
        "default": '`"polynomial"`',
    }
    dict[f"{prof}_coefs"] = {
        "category": ["Initialization", "profiles"],
        # "subtitle": ,
        "description": [
            f"Depending on the type of profile of the {proflong} `{prof}(s)`:",
            f'- if lnk_to_param(`{prof}_type="polynomial"`|{prof}_type), it sets the coefficients $c_0 + c_1 s + c_2 s^2 \dots$',
            f'- if lnk_to_param(`{prof}_type="bspline"`|{prof}_type), these are the B-Spline coefficients (which need to be compatible to the knots lnk_to_param({prof}_knots))',
        ],
        "type": "array of `real` of any size",
        "required_if": f'lnk_to_param({prof}_type) is `"polynomial"` or `"bspline"`',
    }
    dict[f"{prof}_knots"] = {
        "category": ["Initialization", "profiles"],
        # "subtitle":
        "description": [
            f"Required for the B-Spline profile of the {proflong} `{prof}(s)`:",
            "The knots must be in the range of $s=[0,1]$ and must be monotonically increasing.",
            "They must have the same multiplicity of the first knot at $s=0$ and the last knot at $s=1$. The degree of the B-Spline is deduced from the multiplicity.",
            f"Knots need to be compatible with the B-Spline coefficients lnk_to_param({prof}_coefs)",
        ],
        "type": "array of `real` of any size",
        "required_if": f'lnk_to_param({prof}_type) is `"bspline"`',
    }
    dict[f"{prof}_rho2"] = {
        "category": ["Initialization", "profiles"],
        # "subtitle":
        "description": [
            f"Required for the cubic spline interpolation of the {proflong} `{prof}(s)`:",
            "These are the radial point positions, in the normalized magnetic flux",
            "They must cover the range of $s=[0,1]$ and must be monotonically increasing.",
            f"Point positions must of same size as the values, specified by lnk_to_param({prof}_vals)",
        ],
        "type": "array of `real` of any size",
        "required_if": f'lnk_to_param({prof}_type) is `"interpolation"`',
    }
    dict[f"{prof}_vals"] = {
        "category": ["Initialization", "profiles"],
        # "subtitle": True
        "description": [
            f"Required for the cubic spline interpolation of the {proflong} `{prof}(s)`:",
            f"These are the values of `{prof}` at the radial point positions, in the normalized magnetic flux",
            f"Point positions must of same size as the values, specified by lnk_to_param({prof}_rho2)",
        ],
        "type": "array of `real` of any size",
        "required_if": f'lnk_to_param({prof}_type) is `"interpolation"`',
    }
    dict[f"{prof}_BC_type_axis"] = {
        "category": ["Initialization", "profiles"],
        # "subtitle": True
        "description": [
            f"Boundary condition at $s=0$ for the cubic spline interpolation of the {proflong} `{prof}(s)`",
        ],
        "type": "`string`",
        "allowed_table": [
            (
                '`"not_a_knot"`',
                "Makes the third derivative continuous at the second grid point",
            ),
            (
                '`"1st_deriv"`',
                f"Sets the first derivative at the boundary $s=0$, either to zero or to the provided first value of lnk_to_param({prof}_BC_vals)",
            ),
            (
                '`"2nd_deriv"`',
                f"Sets the second derivative at the boundary $s=0$, either to zero or to the provided first value of lnk_to_param({prof}_BC_vals)",
            ),
        ],
        "required_if": f'lnk_to_param({prof}_type) is `"interpolation"`',
        "default": '`"not_a_knot"`',
    }
    dict[f"{prof}_BC_type_edge"] = {
        "category": ["Initialization", "profiles"],
        # "subtitle": True
        "description": [
            f"Boundary condition at $s=1$ for the cubic spline interpolation of the {proflong} `{prof}(s)`",
        ],
        "type": "`string`",
        "allowed_table": [
            (
                '`"not_a_knot"`',
                "Makes the third derivative continuous at the second to last grid point",
            ),
            (
                '`"1st_deriv"`',
                f"Sets the first derivative at the boundary $s=1$, either to zero or to the provided second value of lnk_to_param({prof}_BC_vals)",
            ),
            (
                '`"2nd_deriv"`',
                f"Sets the second derivative at the boundary $s=0$, either to zero or to the provided second value of lnk_to_param({prof}_BC_vals)",
            ),
        ],
        "required_if": f'lnk_to_param({prof}_type) is `"interpolation"`',
        "default": '`"not_a_knot"`',
    }
    dict[f"{prof}_BC_vals"] = {
        "category": ["Initialization", "profiles"],
        # "subtitle": True
        "description": [
            f"Values for the first / second derivative, at $s=0$ and $s=1$, for the cubic spline interpolation of the {proflong} `{prof}(s)`.",
        ],
        "type": "array of `real` of size 2",
        "required_if": f'lnk_to_param({prof}_type) is `"interpolation"` , and if lnk_to_param({prof}_BC_type_axis) or lnk_to_param({prof}_BC_type_edge) is different from `"not_a_knot"`',
        "default": "`(/0.0,0.0/))`",
    }
    dict[f"{prof}_scale"] = {
        "category": ["Initialization", "profiles"],
        # "subtitle": "optional"
        "description": [
            "Scales the {proflong} `{prof}(s)` by a constant.",
            f"See profile definition lnk_to_param({prof}_type)",
        ],
        "type": "`real`",
        "default": "`1.0`",
    }

# pres,iota


with open("parameters.yaml", "w") as f:
    yaml.safe_dump(dict, f, allow_unicode=True, sort_keys=False)

# with open("parameters.yaml", "r") as f:
#    dict_check = yaml.safe_load(f)

# print(dict.keys())
# print(dict_check.keys())

In [ ]:
from generate_parameter_list import format_parameter_list

format_parameter_list(
    "parameters.yaml",
    output_file="parameters-all.md",
    formatting="markdown",
)

In [ ]:
# this has to fail, prevents the next cells to be executed
assert False

# FIRST ATTEMPT

In [ ]:
# functions to write the dropdown + card in markdown


def block(nlevel, command, title, description, opt=None):
    """
    writes the description in a markdown block, with a header with leading ":"nlevel  and a footer with ":"*nlevel. the command and title are in the header, the option `opt` is possible to add after the header, then the description is added (must be a list of strings. Output is a list of strings for each line.
    """
    outstr = []
    outstr.append(":" * nlevel + "{" + command + "} " + title)
    if opt:
        outstr.append(opt)
    # outstr.append('<'*nlevel) #debug
    outstr.append("")
    [outstr.append(x) for x in description]
    # outstr.append('>'*nlevel)#debug
    outstr.append("")
    outstr.append(":" * nlevel)
    return outstr


def generate_doc_param(param, vals):
    """
    generate the markdown documentation for one parameter, using a dropdown with cards inside.
    """
    descr = []
    [descr.append(x) for x in vals["Description"]]
    descr.append("")
    descr.append("**Options:** ")
    descr.append("| | |")
    descr.append("| :-- | :-- |")
    for opt, exp in vals["Options"].items():
        descr.append(f"| `{opt}` | {exp} |")

    top_content = block(5, "grid-item-card", f"`{param}`", descr)

    top_block = block(6, "grid", "1 1 1 1", top_content)

    col_left_a = block(3, "grid-item-card", "Dependency", vals["Dependency"])
    col_left_block = block(4, "grid", "1 1 1 1", col_left_a, opt=":gutter: 1")
    col_left = block(5, "grid-item", "", col_left_block)

    col_right_a = block(3, "grid-item-card", "Proposal", vals["Proposal"])

    typ = [f"`{vals['Datatype']}`, {vals['Size']}"]
    col_right_b = block(3, "grid-item-card", "Datatype", typ)
    col_right_content = col_right_a
    col_right_content.append("")
    [col_right_content.append(x) for x in col_right_b]
    col_right_block = block(4, "grid", "1 1 1 1", col_right_content, opt=":gutter: 1")
    col_right = block(5, "grid-item", "", col_right_block)
    two_cols_content = col_left
    two_cols_content.append("")
    [two_cols_content.append(x) for x in col_right]
    two_cols = block(6, "grid", "1 1 2 2", two_cols_content, opt=":gutter: 1")

    allblocks = top_block
    allblocks.append("")
    [allblocks.append(x) for x in two_cols]

    return block(7, "dropdown", f"`{param}`", allblocks)

In [ ]:
dict = {}

dict["MinimizerType"] = {
    "Description": [r"so it is, $a/b=c$ and so it goes on."],
    "Options": {"0": r"gradient descent in $X^1$", "10": r"accel. in $X^2$"},
    "Datatype": "integer",
    "Size": "scalar",
    "Dependency": [
        r"* is only active if $x_2=x_1$ ",
        r"* has nothing else",
        r"* or one more thing $k=1$",
    ],
    "Proposal": [r" if not provided, set `=LA_deg` "],
}
dict["MinimizerTol"] = {
    "Description": [r"so it is, $a/b=c$", "", r" and so it goes on."],
    "Options": {"0": r"gradient descent in $X^1$", "10": r"accel. in $X^2$"},
    "Datatype": "integer",
    "Size": "scalar",
    "Dependency": [
        r"* is only active if $x_2=x_1$ ",
        r"* has nothing else",
        r"* or one more thing $k=1$",
    ],
    "Proposal": [r" if not provided, set `=LA_deg` "],
}

In [ ]:
# loop over all parameters in dict
outstring = []
for key, vals in dict.items():
    tmpstr = generate_doc_param(key, vals)
    [outstring.append(x) for x in tmpstr]
    outstring.append("")

with open("../user/minimizer-parameters.md", "w") as f:
    for line in outstring:
        f.write(line + "\n")